# Solución Caso 1

## Conjuntos Necesarios

In [11]:
import pandas as pd
#Clientes y sus datos
clients_df = pd.read_csv('data/Clients.csv')
clients_dict = {f"C{int(row['ClientID'])}": [int(row['Product-Type-A']), int(row['Product-Type-B']), int(row['Product-Type-C']),row['Longitude'],row['Latitude']] for _, row in clients_df.iterrows()}
print(clients_dict)

#Depositos y sus datos
deposits_df = pd.read_csv('data/Depots_With_Products.csv')
deposits_dict = {f"D{int(row['DepotID'])}": [int(row['DepotID']),row['Longitude'],row['Latitude'], int(row['Product-Type-A']), int(row['Product-Type-B']), int(row['Product-Type-C'])] for _, row in deposits_df.iterrows()}
print(deposits_dict)

#Vehículos y sus datos
vehicles_df = pd.read_csv('data/Vehicles.csv')
type_mapping = {
    'gas car': 'GS',
    'ev': 'EV',
    'solar ev': 'EV',
    'drone': 'DR'
}
vehicles_dict = {}
for idx, row in enumerate(vehicles_df.itertuples(), start=1):
    vehicle_id = f"V{idx}"  # Claves como V1, V2, ...
    vehicles_dict[vehicle_id] = [type_mapping[row.VehicleType.lower()], row.Capacity, row.Range]
print(vehicles_dict)

#Datos de Costos por tipo de vehículo
vehicles_data_df = pd.read_csv('data/vehicles_data.csv')
vehicles_data_df.rename(
    columns={
        'Vehicle': 'Type',
        'Freight Rate [COP/km]': 'TF',
        'Time Rate [COP/min]': 'TT',
        'Daily Maintenance [COP/day]': 'CM',
        'Recharge/Fuel Cost [COP/(gal or kWh)]': 'CRC',
        'Recharge/Fuel Time [min/10 percent charge]': 'TR',
        'Avg. Speed [km/h]': 'VP',
        'Gas Efficiency [km/gal]': 'Gas_Efficiency',
        'Electricity Efficency [kWh/km]': 'Electricity_Efficiency'
    },
    inplace=True
)

vehicles_parameters = {}
for _, row in vehicles_data_df.iterrows():
    vehicle_type = type_mapping[row['Type'].lower()]
    vehicles_parameters[vehicle_type] = {
        'TF': row['TF'],
        'TT': row['TT'],
        'CM': row['CM'],
        'CRC': row['CRC'],
        'TR': row['TR'],
        'VP': row['VP'],
        'EC': row['Gas_Efficiency'] if not pd.isna(row['Gas_Efficiency']) else row['Electricity_Efficiency']
    }
print(vehicles_parameters)

#Matriz de distancias terrestres
distance_matrix_df = pd.read_csv('data/distance_matrix.csv', index_col=0)
# distance_matrix_df = pd.read_csv('distance_single_matrix.csv', index_col=0)
distance_dict = {
    (row, col): float(distance_matrix_df.loc[row, col])
    for row in distance_matrix_df.index
    for col in distance_matrix_df.columns
}
print({key: distance_dict[key] for key in list(distance_dict.keys())[:20]})

#Matriz de tiempos terrestres
duration_matrix_df = pd.read_csv('data/duration_matrix.csv', index_col=0)
# duration_matrix_df = pd.read_csv('duration_single_matrix.csv', index_col=0)
duration_dict = {
    (row, col): float(duration_matrix_df.loc[row, col])
    for row in duration_matrix_df.index
    for col in duration_matrix_df.columns
}
print({key: duration_dict[key] for key in list(duration_dict.keys())[:20]})

#Matriz de distancias aéreas
distance_air_df = pd.read_csv('data/distance_dron_matrix.csv', index_col=0)
# distance_air_df = pd.read_csv('distance_dron_single_matrix.csv', index_col=0)
distance_air_dict = {
    (row, col): float(distance_air_df.loc[row, col])
    for row in distance_air_df.index
    for col in distance_air_df.columns
}
print({key: distance_air_dict[key] for key in list(distance_air_dict.keys())[:20]})

{'C1': [350, 550, 0, -74.19699184741948, 4.632552840424734], 'C2': [1, 1, 1, -74.12032773611021, 4.719616298449461], 'C3': [1, 1, 1, -74.11027242131048, 4.727691608175209], 'C4': [1, 1, 1, -74.06815337963502, 4.712202539809157], 'C5': [1, 1, 1, -74.19486224157397, 4.638612189685843], 'C6': [1, 1, 1, -74.12751619352659, 4.630489482968551], 'C7': [1, 1, 1, -74.10178731827096, 4.732421040989568], 'C8': [1, 1, 1, -74.0328033058973, 4.703342234821396], 'C9': [1, 1, 1, -74.08665709611086, 4.574297477559144]}
{'D1': [1, -74.08124218159384, 4.75021190869025, 140, 200, 500], 'D2': [2, -74.10993358606953, 4.5363832206427785, 160, 100, 200], 'D3': [3, -74.03854814565923, 4.792925960208614, 240, 200, 300], 'D4': [4, -74.06706883098641, 4.72167778077445, 160, 500, 100], 'D5': [5, -74.13826337931849, 4.607707046760958, 70, 100, 400], 'D6': [6, -74.12400186370824, 4.650463053612691, 140, 500, 800], 'D7': [7, -74.09561875464892, 4.621911772492814, 170, 600, 200], 'D8': [8, -74.10975623736951, 4.678960

## Modelo

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

# Conjuntos
unique_nodes = set(i for i, j in distance_dict.keys()).union(j for i, j in distance_dict.keys())
model.I = Set(initialize=unique_nodes)  # Nodos (orígenes y destinos)
model.PAIRS = Set(dimen=2, initialize=distance_dict.keys())  # Pares válidos (i, j)
model.K = Set(initialize=vehicles_dict.keys())  # Vehículos
model.D = Set(initialize=deposits_dict.keys())  # Depósitos
model.P = Set(initialize=['A', 'B', 'C'])  # Productos

# Mapeo de productos a índices en las listas
product_indices = {'A': 0, 'B': 1, 'C': 2}

# Parámetros
# Distancia y tiempo entre nodos
model.distance = Param(model.PAIRS, initialize=distance_dict)
model.duration = Param(model.PAIRS, initialize=duration_dict)
model.distance_air = Param(model.PAIRS, initialize=distance_air_dict)

# Parámetros del vehículo
model.TF = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TF'] for k in model.K})
model.TT = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TT'] for k in model.K})
model.CM = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CM'] for k in model.K})
model.CRC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['CRC'] for k in model.K})
model.TR = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['TR'] for k in model.K})
model.VP = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['VP'] for k in model.K})
model.EC = Param(model.K, initialize={k: vehicles_parameters[vehicles_dict[k][0]]['EC'] for k in model.K})
model.R = Param(model.K, initialize={k: vehicles_dict[k][2] for k in model.K})  # Rango

# Capacidad máxima de los vehículos
vehicle_capacity = {k: vehicles_dict[k][1] for k in model.K}

# Demanda de cada producto en cada cliente
def demanda_init(model, i, p):
    if i in clients_dict:
        return clients_dict[i][product_indices[p]]
    else:
        return 0

model.demanda = Param(model.I, model.P, initialize=demanda_init)

# Capacidad de cada depósito para cada producto
def capacidad_deposito_init(model, d, p):
    if d in deposits_dict:
        return deposits_dict[d][3 + product_indices[p]]  # Índices 3,4,5 para A,B,C
    else:
        return 0

model.capacidad_deposito_param = Param(model.D, model.P, initialize=capacidad_deposito_init)

# Variables de decisión
model.x = Var(model.I, model.I, model.K, domain=Binary)  # Si el vehículo k viaja de i a j
model.W = Var(model.K, model.P, domain=NonNegativeReals)  # Peso transportado por vehículo k del producto p
model.u = Var(model.I, model.K, domain=NonNegativeReals)  # Posición secuencial en la ruta para evitar subtours

# Variables auxiliares
model.s = Var(model.K, model.D, domain=Binary)  # Indica si el vehículo k sale del depósito d
model.W_s = Var(model.K, model.D, model.P, domain=NonNegativeReals)  # Linealización de W[k,p] * s[k,d]

# Variables adicionales para asignar entregas de productos a clientes
model.y = Var(model.I, model.P, model.K, domain=NonNegativeReals)  # Cantidad del producto p entregada al cliente i por el vehículo k

# Función objetivo
def total_cost_rule(model):
    # Costo por distancia
    C_total = sum(
        model.TF[k] * (
            model.distance[i, j]/1000 if vehicles_dict[k][0].lower() != 'drone' else model.distance_air[i, j]
        ) * model.x[i, j, k]
        for k in model.K for i in model.I for j in model.I if i != j and (i, j) in model.PAIRS
    )
    
    # Costo por tiempo
    C_tiempo = sum(
        model.TT[k] * (
            (model.distance[i, j] / 1000 / (model.VP[k] / 60)) if vehicles_dict[k][0].lower() == 'drone' else (model.duration[i, j] / 60)
        ) * model.x[i, j, k]
        for k in model.K for i in model.I for j in model.I if i != j and (i, j) in model.PAIRS
    )
    
    # Costo por peso transportado
    C_carga = sum(
        sum(model.W[k, p] * 500 for p in model.P)
        for k in model.K
    )
    
    # Costo de mantenimiento diario
    C_mantenimiento = sum(
    model.CM[k] * sum(
        model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j
    )
    for k in model.K
)
    
    # Costo total
    return C_total + C_tiempo + C_carga + C_mantenimiento

model.obj = Objective(rule=total_cost_rule, sense=minimize)

# Restricciones

# 5.1. Salida desde exactamente un depósito
def salida_desde_deposito_rule(model, k):
    return sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j) == 1
model.salida_desde_deposito = Constraint(model.K, rule=salida_desde_deposito_rule)

# Regreso al depósito de origen
def regreso_a_deposito_rule(model, k):
    return sum(model.x[j, d, k] for d in deposits_dict.keys() for j in model.I if d != j) == sum(model.x[d, j, k] for d in deposits_dict.keys() for j in model.I if d != j)
model.regreso_a_deposito = Constraint(model.K, rule=regreso_a_deposito_rule)

# 5.3. Satisfacción de la demanda de los clientes por producto
def satisfacer_demanda_rule(model, i, p):
    if i in clients_dict:
        return sum(model.y[i, p, k] for k in model.K) == model.demanda[i, p]
    else:
        return Constraint.Skip
model.satisfacer_demanda = Constraint(model.I, model.P, rule=satisfacer_demanda_rule)

# 5.4. Relacionar y con x mediante restricciones lineales

# Restricción 1: y[i,p,k] <= W[k,p]
def relacionar_y_con_W_rule(model, i, p, k):
    if i in clients_dict:
        return model.y[i, p, k] <= model.W[k, p]
    else:
        return Constraint.Skip
model.relacionar_y_con_W = Constraint(model.I, model.P, model.K, rule=relacionar_y_con_W_rule)

# Restricción 2: y[i,p,k] <= demanda[i,p] * sum(x[j,i,k] for j in model.I if (j,i) in model.PAIRS)
def relacionar_y_con_x_rule_improved(model, i, p, k):
    if i in clients_dict:
        return model.y[i, p, k] <= model.demanda[i, p] * sum(model.x[j, i, k] for j in model.I if (j, i) in model.PAIRS)
    else:
        return Constraint.Skip
model.relacionar_y_con_x_improved = Constraint(model.I, model.P, model.K, rule=relacionar_y_con_x_rule_improved)

# 5.5. Flujo correcto entre nodos (clientes y depósitos)
def flujo_correcto_rule(model, i, k):
    if i in model.D:
        return Constraint.Skip
    return sum(model.x[i, j, k] for j in model.I if i != j and (i, j) in model.PAIRS) == \
           sum(model.x[j, i, k] for j in model.I if i != j and (j, i) in model.PAIRS)
model.flujo_correcto = Constraint(model.I, model.K, rule=flujo_correcto_rule)

# 5.6. Restricciones MTZ para evitar subtours
def mtz_rule(model, i, j, k):
    if i == j or i in model.D or j in model.D:
        return Constraint.Skip
    n = len(model.I)
    return model.u[i, k] - model.u[j, k] + (n - 1) * model.x[i, j, k] <= n - 2
model.mtz = Constraint(model.I, model.I, model.K, rule=mtz_rule)

# 5.7. Capacidad máxima de carga por vehículo
def capacidad_carga_rule(model, k):
    return sum(model.W[k, p] for p in model.P) <= vehicle_capacity[k]
model.capacidad_carga = Constraint(model.K, rule=capacidad_carga_rule)

# 5.8. Capacidad y Demanda por producto y vehículo
def capacidad_demanda_rule(model, k, p):
    # Cambiado de >= a <= y relaciona y con W
    return sum(model.y[i, p, k] for i in clients_dict.keys()) <= model.W[k, p]
model.capacidad_demanda = Constraint(model.K, model.P, rule=capacidad_demanda_rule)

# 5.9. Linealización del producto W[k,p] * s[k,d]
def linearizacion_producto_rule1(model, k, d, p):
    return model.W_s[k, d, p] <= model.W[k, p]
model.linearizacion_producto1 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule1)

def linearizacion_producto_rule2(model, k, d, p):
    return model.W_s[k, d, p] <= vehicle_capacity[k] * model.s[k, d]
model.linearizacion_producto2 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule2)

def linearizacion_producto_rule3(model, k, d, p):
    return model.W_s[k, d, p] >= model.W[k, p] - vehicle_capacity[k] * (1 - model.s[k, d])
model.linearizacion_producto3 = Constraint(model.K, model.D, model.P, rule=linearizacion_producto_rule3)

# 5.10. Capacidad del depósito por producto
def capacidad_deposito_rule(model, d, p):
    return sum(model.W_s[k, d, p] for k in model.K) <= model.capacidad_deposito_param[d, p]
model.capacidad_deposito = Constraint(model.D, model.P, rule=capacidad_deposito_rule)

# 5.11. Relacionar s[k,d] con x[d,j,k]
def s_definition_rule(model, k, d):
    # Corregido: eliminar ',k' en la condición
    return model.s[k, d] == sum(model.x[d, j, k] for j in model.I if (d, j) in model.PAIRS)
model.s_definition = Constraint(model.K, model.D, rule=s_definition_rule)

# 5.12. Asegurar que cada vehículo sale de un único depósito
def unica_salida_deposito_rule(model, k):
    return sum(model.s[k, d] for d in model.D) == 1
model.unica_salida_deposito = Constraint(model.K, rule=unica_salida_deposito_rule)

# 5.13. Distancia Máxima (Rango) para cada vehículo
def distancia_maxima_rule(model, k):
    return sum(
        (model.distance[i, j]/1000 if vehicles_dict[k][0].lower() != 'drone' else model.distance_air[i, j]) * model.x[i, j, k]
        for i in model.I for j in model.I if i != j and (i, j) in model.PAIRS
    ) <= model.R[k]
model.distancia_maxima = Constraint(model.K, rule=distancia_maxima_rule)

# 5.14. Carga total de productos desde depósitos
def carga_total_from_depositos_rule(model, k, p):
    return sum(model.W_s[k, d, p] for d in model.D) == model.W[k, p]
model.carga_total_from_depositos = Constraint(model.K, model.P, rule=carga_total_from_depositos_rule)


(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo


In [13]:
solver_name = "appsi_highs"
solver = SolverFactory(solver_name)
solver.options['parallel'] = 'on'
solver.options['threads'] = 6
solver.options['time_limit'] = 600  # 1-hour time limit
solver.options['mip_rel_gap'] = 0.05  # 5% relative gap
result = solver.solve(model, tee=True)

Running HiGHS 1.8.1 (git hash: 4a7f24a): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+03]
  Cost   [5e+02, 2e+05]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+03]
Presolving model
1645 rows, 2518 cols, 13316 nonzeros  0s
1432 rows, 1645 cols, 7916 nonzeros  0s
1355 rows, 1592 cols, 7335 nonzeros  0s

Solving MIP model with:
   1355 rows
   1592 cols (1151 binary, 0 integer, 0 implied int., 441 continuous)
   7335 nonzeros
MIP-Timing:        0.15 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol    

## Resultados

### Texto Plano

In [14]:
results_path = "results/results2.txt"

with open(results_path, "w") as file:
    # Escribir el costo total
    file.write("=== Resultados ===\n")
    file.write(f"Costo total operativo: {model.obj():.2f} COP\n\n")
    
    # Escribir los recorridos por vehículo
    file.write("Recorridos asignados por vehículo:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        recorrido = []
        for i in model.I:
            for j in model.I:
                if i != j and model.x[i, j, k].value > 0.5:  # Verifica si el vehículo viaja de i a j
                    recorrido.append((i, j))
        if recorrido:
            recorrido_str = " -> ".join([f"{i} a {j}" for i, j in recorrido])
            file.write(f"  Recorrido: {recorrido_str}\n")
        else:
            file.write("  No tiene asignaciones.\n")
    
    # Escribir distancias totales recorridas
    file.write("\nDistancias totales recorridas:\n")
    for k in model.K:
        # Calcular la distancia total recorrida por el vehículo k
        distancia_total = sum(
            (model.distance[i, j]/1000 if vehicles_dict[k][0] != 'DR' else model.distance_air[i, j]) * model.x[i, j, k].value
            for i in model.I for j in model.I if i != j and (i, j) in model.PAIRS
        )
        file.write(f"  Vehículo {k}: {distancia_total:.2f} km\n")
    
    # Escribir pesos totales transportados por vehículo y por producto
    file.write("\nPesos totales transportados por vehículo y por producto:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        for p in model.P:
            peso = model.W[k, p].value
            file.write(f"  Producto {p}: {peso:.2f} kg\n")
    
    # Escribir pesos totales transportados por vehículo desde cada depósito y por producto (si es necesario)
    # Puedes descomentar y ajustar esta sección si deseas incluirlo
    
    file.write("\nPesos transportados por vehículo desde cada depósito y por producto:\n")
    for k in model.K:
        file.write(f"\nVehículo {k}:\n")
        for d in model.D:
            for p in model.P:
                peso_s = model.W_s[k, d, p].value
                if peso_s > 0:
                    file.write(f"  Desde Depósito {d}, Producto {p}: {peso_s:.2f} kg\n")
    
    
    # Escribir costos adicionales si es necesario (opcional)
    
    # Ejemplo: Costo por peso transportado
    file.write("\nCosto por peso transportado:\n")
    for k in model.K:
        costo_peso = sum(model.W[k, p].value * 500 for p in model.P)
        file.write(f"  Vehículo {k}: {costo_peso:.2f} COP\n")
    
    
print(f"Resultados escritos en el archivo: {results_path}")


Resultados escritos en el archivo: results/results2.txt


### Mapa Interactivo

In [9]:
import folium
from folium.plugins import AntPath
import requests

# Crear un mapa centrado en una ubicación aproximada
centro_lat = deposits_df['Latitude'].mean()
centro_lon = deposits_df['Longitude'].mean()
mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

# Asignar colores según el tipo de vehículo
tipo_colores = {
    "GS": "red",  # Gasoline cars
    "EV": "blue",  # Electric vehicles
    "DR": "green"  # Drones
}

# Agregar marcadores para depósitos
for depot_id, (id, lon, lat, proda,prodb,proc) in deposits_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Depósito {depot_id}",
        icon=folium.Icon(color="blue", icon="home")
    ).add_to(mapa)

# Agregar marcadores para clientes
for client_id, (producta,productb,productc, lon, lat) in clients_dict.items():
    folium.Marker(
        location=[lat, lon],
        popup=f"Cliente {client_id} - Producto {producta}",
        icon=folium.Icon(color="green", icon="user")
    ).add_to(mapa)

# Función para calcular ruta real con OSRM
def calcular_ruta_osrm(coord_inicio, coord_fin):
    try:
        url = f"http://router.project-osrm.org/route/v1/driving/{coord_inicio[1]},{coord_inicio[0]};{coord_fin[1]},{coord_fin[0]}"
        params = {"overview": "full", "geometries": "geojson"}
        response = requests.get(url, params=params)
        if response.status_code == 200:
            route = response.json()
            geometry = route["routes"][0]["geometry"]["coordinates"]
            return [(lat, lon) for lon, lat in geometry]
        else:
            print(f"Error OSRM: {response.status_code}")
            return [coord_inicio, coord_fin]
    except Exception as e:
        print(f"Error al calcular ruta OSRM: {e}")
        return [coord_inicio, coord_fin]

# Dibujar los recorridos de los vehículos con estilos según el tipo
for k in model.K:
    tipo_vehiculo = vehicles_dict[k][0]  # Obtener el tipo de vehículo (GS, EV, DR)
    color = tipo_colores[tipo_vehiculo]  # Asignar color según el tipo

    for i in model.I:
        for j in model.I:
            if i != j and model.x[i, j, k].value > 0.5:  # Si hay un recorrido entre i y j
                # Obtener coordenadas de los nodos
                if i in clients_dict:
                    coord_i = (clients_dict[i][2], clients_dict[i][1])  # (lat, lon)
                else:
                    coord_i = (deposits_dict[i][2], deposits_dict[i][1])  # (lat, lon)
                if j in clients_dict:
                    coord_j = (clients_dict[j][2], clients_dict[j][1])  # (lat, lon)
                else:
                    coord_j = (deposits_dict[j][2], deposits_dict[j][1])  # (lat, lon)

                # Calcular la ruta (línea recta para drones, ruta OSRM para otros)
                if tipo_vehiculo == "DR":
                    ruta = [coord_i, coord_j]  # Línea recta
                else:
                    ruta = calcular_ruta_osrm(coord_i, coord_j)  # Camino real

                #print(ruta)

                # Dibujar la línea
                AntPath(
                    locations=ruta,
                    color=color,
                    weight=2.5,
                    opacity=1,
                    dash_array=[1, 20],  # Movimiento solo para drones
                    tooltip=f"Vehículo {k} ({tipo_vehiculo}): {i} -> {j}"
                ).add_to(mapa)

# Guardar el mapa en un archivo HTML
mapa.save("results/recorridos_por_tipo.html")
print("Mapa guardado como 'recorridos_por_tipo.html'")


Mapa guardado como 'recorridos_por_tipo.html'


### PDF de Reporte

In [ ]:
!pip install fpdf

12


In [ ]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'Informe de Rutas Optimizadas', border=0, ln=True, align='C')
        self.ln(10)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 10)
        self.cell(0, 10, f'Página {self.page_no()}', align='C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, ln=True, align='L')
        self.ln(5)

    def chapter_body(self, body):
        self.set_font('Arial', '', 11)
        self.multi_cell(0, 10, body)
        self.ln()

    def add_table(self, header, data, col_widths):
        self.set_font('Arial', 'B', 11)
        for i, col in enumerate(header):
            self.cell(col_widths[i], 10, col, border=1, align='C')
        self.ln()
        self.set_font('Arial', '', 10)
        for row in data:
            for i, item in enumerate(row):
                if isinstance(item, str) and item.startswith("http"):  # Enlace clickeable
                    self.set_text_color(0, 0, 255)
                    self.cell(col_widths[i], 10, 'Ver recorrido', border=1, align='C', link=item)
                    self.set_text_color(0, 0, 0)
                else:
                    self.cell(col_widths[i], 10, str(item), border=1, align='C')
            self.ln()

# Crear el PDF
pdf = PDF()
pdf.add_page()

# Título del informe
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Resultados del Modelo de Optimización', align='C', ln=True)
pdf.ln(10)

# Resumen general
pdf.chapter_title('Resumen de Resultados:')
summary = (
    "Costo total operativo: 807,789.26 COP\n\n"
    "Distancias totales recorridas (km):\n"
    "  - Vehículo V5: 21.45 km\n"
    "  - Vehículo V6: 35.14 km\n"
    "  - Vehículo V7: 20.10 km\n"
    "  - Vehículo V8: 7.84 km\n"
    "  - Vehículo V9: 9.12 km\n"
    "  - Vehículo V10: 2.10 km\n"
    "  - Vehículo V11: 14.09 km\n"
    "  - Vehículo V12: 2.85 km\n"
)
pdf.chapter_body(summary)

# Información de vehículos
pdf.chapter_title('Pesos Totales Transportados por Vehículo (kg):')
header = ['Vehículo', 'Peso Transportado']
data = [
    ['V5', '77.00'], ['V6', '78.00'], ['V7', '69.00'],
    ['V8', '36.00'], ['V9', '45.00'], ['V10', '18.00'],
    ['V11', '32.00'], ['V12', '22.00']
]
pdf.add_table(header, data, [50, 50])

# Rutas por vehículo con un enlace único por recorrido
pdf.chapter_title('Recorridos Asignados por Vehículo (Enlace único):')
for vehicle, path in routes.items():
    # Construir la lista de coordenadas en orden
    coordinates = []
    for start, end in path:
        if not coordinates or coordinates[-1] != start:
            coord_start = (clients_dict[start][2], clients_dict[start][1]) if start in clients_dict else (deposits_dict[start][2], deposits_dict[start][1])
            coordinates.append(coord_start)
        coord_end = (clients_dict[end][2], clients_dict[end][1]) if end in clients_dict else (deposits_dict[end][2], deposits_dict[end][1])
        coordinates.append(coord_end)
    
    # Crear el enlace de Google Maps con múltiples paradas
    base_url = "https://www.google.com/maps/dir/"
    map_link = base_url + "/".join([f"{lat:.6f},{lon:.6f}" for lat, lon in coordinates])

    # Agregar el vehículo y su enlace al PDF
    pdf.chapter_title(f'Vehículo {vehicle} ({vehicles_dict[vehicle][0]}):')
    pdf.chapter_body(f"Enlace a la ruta completa: ")
    pdf.cell(0, 10, 'Ver recorrido', ln=True, link=map_link)

# Guardar el PDF
pdf_path = "results/informe_rutas_viaje_unico.pdf"
pdf.output(pdf_path)
print(f"Informe generado: {pdf_path}")
